# 3. Data Preparation

In this phase we will prepare the dataset, modifying it according to the previous phase. In the end of this notebook we should have a dataset ready to be used in modelling.

## 3.1 Select Data

We begin by loading the complete raw data so that we have more flexibility in the preparation of the data.

The selection of the most relevant variables will be done in the next steps.

In [ ]:
import pandas as pd

student = pd.read_csv("../../../databases/2018/student2018.csv")
teacher = pd.read_csv("../../../databases/2018/only_teacher2018.csv")

#student = pd.read_csv("../../../databases/2018/student2018.csv", nrows=2000)
#teacher = pd.read_csv("../../../databases/2018/only_teacher2018.csv", nrows=2000)

print("student shape:", student.shape)
print("teacher shape:", teacher.shape)

## 3.2 Clean Data

From the previous phase, we decided to remove unnecessary variables according the following criteria:
- Variables with more than 70% missing values.
- Students and teachers from England, due to differences in their education system.

#### Students data

In [ ]:
missing = student.isnull().mean().sort_values(ascending=False)
student = student.drop(columns=missing[missing > 0.7].index)

student = student[(student["CNT"] != "b'GBR'")]

student.head(5)


**List of countries**:
teacher dataset only has these countries (so we should remove all students from other countries):
`ALB`, `QAZ`, `BRA`, `CHL`, `TAP`, `DOM`, `DEU`, `HKG`, `KOR`, `MAC`,`MYS`,`MAR`,`PAN`,`PER`,`PRT`,`ESP`,`ARE`,`GBR`,`USA`

In [ ]:
student = student[
    student["CNT"].isin(teacher['CNT'].unique())
]

print(student['CNT'].unique())

#### Teachers data

In [ ]:
missing = teacher.isnull().mean().sort_values(ascending=False)
teacher = teacher.drop(columns=missing[missing > 0.7].index)

teacher = teacher[(teacher["CNT"] != "b'GBR'")]

teacher.head(5)

del missing

In [ ]:
print("student shape:", student.shape)
print("teacher shape:", teacher.shape)

## 3.3 Construct Data

In this step, we construct new variables based on the available data.

For each subject, there are 10 plausible values (PVs), which we averaged into a single score column per subject.

In [ ]:
from columns_list_q2 import (
    get_avg_results,
    drop_columns,
    math_columns,
    reading_columns,
    science_columns
)

student = get_avg_results(student, reading_columns, "Reading", None)
student = get_avg_results(student, math_columns, "Math", None)
student = get_avg_results(student, science_columns, "Science", None)

student = drop_columns(student)

print(pd.to_numeric(student['Avg Reading Result'], errors='coerce').isna().sum())
print(pd.to_numeric(student['Avg Science Result'], errors='coerce').isna().sum())
print(pd.to_numeric(student['Avg Math Result'], errors='coerce').isna().sum())

del math_columns
del reading_columns
del science_columns

In [ ]:
student.head(5)

In [ ]:
print("student shape:", student.shape)
print("teacher shape:", teacher.shape)

## 3.4 Integrate Data

First save data, to keep memory clean

In [ ]:
teacher.to_csv("../../../databases/2018/cleaned_teacher2018.csv", index=False)
student.to_csv("../../../databases/2018/cleaned_students2018.csv", index=False)

del teacher
del student

#### Join tables

In [ ]:
"""
columns_to_keep = [
    'Avg Math Result', 'Avg Science Result', 'Avg Reading Result"'
    'CNT', 'CNTSCHID', 'REPEAT', 'ST001D01T', 'CNTSTUID'
]
"""

# Function to load and merge datasets in chunks
def init_target_csv():
    with open("../../../databases/2018/cleaned_teacher_student2018.csv", 'w') as file:
        pass # Do nothing, just open and close the file

    df1 = pd.read_csv('../../../databases/2018/cleaned_students2018.csv', nrows=2)
    df2 = pd.read_csv('../../../databases/2018/cleaned_teacher2018.csv', nrows=2)

    # df1 = df1[columns_to_keep]

    df_result = pd.DataFrame(columns=(df1.columns.append(df2.columns)).unique())
    df_result.to_csv("../../../databases/2018/cleaned_teacher_student2018.csv", index_label=False)

# Function to load and merge datasets in chunks
def get_dataset_in_chunks(chunk_size):
    chunks = []
    i=0
    for chunk in pd.read_csv('../../../databases/2018/cleaned_students2018.csv', chunksize=chunk_size):
        print("i=",i,flush=True)
        i=i+1

        #chunk = chunk[columns_to_keep]

        merged_chunk = pd.merge(chunk, pd.read_csv('../../../databases/2018/cleaned_teacher2018.csv'), on=['CNT', 'CNTSCHID'])
        merged_chunk.to_csv("../../../databases/2018/cleaned_teacher_student2018.csv", mode="a", header=False, index=False)
        #print(merged_chunk.head(), flush=True)
        #chunks.append(merged_chunk)
    #return pd.concat(chunks, ignore_index=True)

# Define the chunk size
chunk_size = 50000 # Adjust the chunk size based on your memory capacity

init_target_csv()
# Perform the merge in chunks
get_dataset_in_chunks(chunk_size)

print("Merge completed and saved to 'cleaned_teacher_student2018.csv'.")

In [ ]:
import pandas as pd

#load new table
teacher_student = pd.read_csv('../../../databases/2018/cleaned_teacher_student2018.csv')


In [ ]:
# Group by teacher (CNTTCHID) with students performance aggregated
summary = teacher_student.groupby('CNTTCHID').agg(
    total_students=('CNTSTUID', 'count'),
    repeating_students=('REPEAT', 'sum'),
    highest_reading_score=('Avg Reading Result', 'max'),
    lowest_reading_score=('Avg Reading Result', 'min'),
    average_reading_score=('Avg Reading Result', 'mean'),
    average_science_score=('Avg Science Result', 'mean'),
    average_math_score=('Avg Math Result', 'mean')
).reset_index()

summary['percentage_repeating'] = (summary['repeating_students'] / summary['total_students']) * 100

# Merge statistics back into the original teacher with student data aggregated
teacher_student = teacher_student.merge(
    summary[[
        'CNTTCHID',
        'percentage_repeating',
        'total_students',
        'repeating_students',
        'highest_reading_score',
        'lowest_reading_score',
        'average_reading_score',
        'average_science_score',
        'average_math_score'
    ]],
    on='CNTTCHID', how='left')

teacher_student.drop(columns=['Avg Reading Result', 'Avg Science Result', 'Avg Math Result', 'REPEAT'])
teacher_student = teacher_student.groupby('CNTTCHID')

#Save database with aggregated data
teacher_student.to_csv("../../../databases/2018/integrated_teacher_student2018.csv", index_label=False)
del summary
del teacher_student


## 3.5 Format Data

In the previous phase we concluded that Country variable should be grouped into fewer values to avoid high dimensionality.

To reduce the cardinality of the "CNT" (Country) variable while preserving meaningful educational information, countries were grouped into three categories based on their average PISA performance: Above Average, Average, and Below Average.

In [ ]:
#load new table
teacher_student = pd.read_csv('../../../databases/2018/integrated_teacher_student2018.csv')

country_avg_scores = teacher_student.groupby("CNT")["average_reading_score"].mean()
tolerance = 15
global_avg = country_avg_scores.mean()

def categorize_country(score):
    if score >= global_avg + tolerance:
        return "Above Average"
    elif score <= global_avg - tolerance:
        return "Below Average"
    else:
        return "Average"

country_performance_group = country_avg_scores.apply(categorize_country)

teacher_student["CNT_Group"] = teacher_student["CNT"].map(country_performance_group)
teacher_student["CNT_Group"] = teacher_student["CNT_Group"].astype("category")

print(teacher_student["CNT_Group"].value_counts())

In [ ]:
def categorize_reading_teacher_by_teacher_performance(df):
    # Sort the DataFrame by 'Avg Reading Result' in descending order
    df = df.sort_values(by='average_reading_score', ascending=False).reset_index(drop=True)

    # Calculate the number of students
    total = len(df)

    # Define the percentage thresholds
    very_good_threshold = int(total * 0.90)
    good_threshold = int(total * 0.70)
    sufficient_threshold = int(total * 0.50)

    # Assign categories from highest to lowest
    df.loc[0:sufficient_threshold-1, 'Reading Score Classification'] = 'Insufficient (0-49%)'
    df.loc[sufficient_threshold:good_threshold-1, 'Reading Score Classification'] = 'Sufficient (50-69%)'
    df.loc[good_threshold:very_good_threshold-1, 'Reading Score Classification'] = 'Good (70-89%)'
    df.loc[very_good_threshold:, 'Reading Score Classification'] = 'Very Good (90-100%)'

    return df

teacher_student = categorize_reading_teacher_by_teacher_performance(teacher_student)

print("Number of students by classification")
print("Very Good (90-100%): ", len(teacher_student[teacher_student['Reading Score Classification']=="Very Good (90-100%)"]))
print("Good (70-89%): ", len(teacher_student[teacher_student['Reading Score Classification']=="Good (70-89%)"]))
print("Sufficient (50-69%): ", len(teacher_student[teacher_student['Reading Score Classification']=="Sufficient (50-69%)"]))
print("Insufficient (0-49%): ", len(teacher_student[teacher_student['Reading Score Classification']=="Insufficient (0-49%)"]))

## Correlation chart

Using a more cleaner dataset, we can check again the correlations between the dataset features and the target variable (average math result).

In [ ]:
correl = (
    teacher_student.corr(numeric_only=True)["average_reading_score"]
    .abs()
    .sort_values(ascending=False)
)

very_good_corr = teacher_student[teacher_student['Reading Score Classification']=="Very Good (90-100%)"].corr(numeric_only=True)[
    "average_reading_score"
].sort_values(ascending=False)
best_performance.drop("average_reading_score")

best_performance = teacher_student[teacher_student['Reading Score Classification']=="Very Good (90-100%)"].corr(numeric_only=True)[
    "average_reading_score"
].sort_values(ascending=False)
best_performance.drop("average_reading_score")


repeating_correlations = repeating_correlations.drop("Avg Math Result")

non_repeating_correlations = non_repeating_students.corr(numeric_only=True)[
    "Avg Math Result"
].sort_values(ascending=False)

non_repeating_correlations = non_repeating_correlations.drop("Avg Math Result")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from dictionary import column_mapping

repeating_correlations = repeating_correlations[repeating_correlations.abs() > 0.3]
non_repeating_correlations = non_repeating_correlations[
    non_repeating_correlations.abs() > 0.3
]
repeating_correlations.index = [
    column_mapping.get(col, col) for col in repeating_correlations.index
]
non_repeating_correlations.index = [
    column_mapping.get(col, col) for col in non_repeating_correlations.index
]

plt.figure(figsize=(12, 8))

sns.barplot(x=repeating_correlations.values, y=repeating_correlations.index)
plt.title("Math result correlation (Repeating Students)")
plt.ylabel("Numeric Variables")
plt.show()

plt.figure(figsize=(12, 8))

sns.barplot(x=non_repeating_correlations.values, y=non_repeating_correlations.index)
plt.title("Math result correlation (Non-Repeating Students)")
plt.ylabel("Numeric Variables")

plt.show()

<!-- Since the focus is on repeating students, we prioritized features that show a strong correlation with this group. -->

Following the correlation analysis, we opted to incorporate variables identified as significant from both the repeating and non-repeating student groups. This mixed feature selection strategy aims to leverage complementary insights from both subsets, enhancing the robustness of the final dataset.

## 3.1 Select Data (Cont.)

After completing the data analysis, we merged the selected numerical and categorical variables to create the final dataset, ready for the next phase.

In [ ]:
from collections import OrderedDict


reversed_column_mapping = {v: k for k, v in column_mapping.items()}
repeating_original_columns = [
    reversed_column_mapping.get(col, col) for col in repeating_correlations.index
]
non_repeating_original_columns = [
    reversed_column_mapping.get(col, col) for col in non_repeating_correlations.index
]

columns_to_include = list(
    OrderedDict.fromkeys(
        repeating_original_columns
        + non_repeating_original_columns
        + ["Avg Math Result", "CNT_Group"]
    )
)

filtered_cols_dataset = filtered_student[columns_to_include].copy()
filtered_cols_dataset = filtered_cols_dataset[sorted(filtered_cols_dataset.columns)]
filtered_cols_dataset

Finally, we analyzed the correlation matrix to check for highly correlated features that might be removed, but none were identified.

In [ ]:
correlation_matrix = filtered_cols_dataset.corr(numeric_only=True)

In [ ]:
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f", square=True)
plt.title("Correlation Matrix")
plt.show()

In [ ]:
filtered_cols_dataset.to_csv(
    "../../../databases/q1_database.csv",
    index=False,
)